In [1]:
import glob, os
from itertools import product
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn import preprocessing

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.neighbors import LocalOutlierFactor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans

import collections 
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold, PredefinedSplit
from sklearn.metrics import accuracy_score
import warnings 


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from pytimekr import pytimekr
warnings.filterwarnings(action="ignore")
%matplotlib inline

# Load the dataset
data = pd.read_csv('./train.csv')
data.head()

plt.rc("font", family="AppleGothic")
plt.rcParams['axes.unicode_minus'] = False


In [2]:
data_test = pd.read_csv('./test.csv')

In [3]:
data['item'].value_counts()

item
TG    15230
BC    13707
RD    12184
CR    10661
CB     7615
Name: count, dtype: int64

In [4]:
data.isna().sum()

ID             0
timestamp      0
item           0
corporation    0
location       0
supply(kg)     0
price(원/kg)    0
dtype: int64

In [5]:
type(data['timestamp'][0])

str

In [6]:
import datetime

data['timestamp'] = pd.to_datetime(data['timestamp'])

# 날짜 --> week와 day column 생성 (day는 요일을 나타냄)

data['year'] = data['timestamp'].dt.isocalendar().year
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['week'] = data['timestamp'].dt.isocalendar().week

data['weekday'] = data['timestamp'].dt.isocalendar().day



In [7]:
import datetime

data_test['timestamp'] = pd.to_datetime(data_test['timestamp'])

# 날짜 --> week와 day column 생성 (day는 요일을 나타냄)

data_test['year'] = data_test['timestamp'].dt.isocalendar().year
data_test['month'] = data_test['timestamp'].dt.month
data_test['day'] = data_test['timestamp'].dt.day
data_test['week'] = data_test['timestamp'].dt.isocalendar().week

data_test['weekday'] = data_test['timestamp'].dt.isocalendar().day



In [8]:
data['isWeekday'] = ((data['weekday'] >= 1) & (data['weekday'] <= 5)).astype(int)
data['isSaturday'] = (data['weekday'] == 6).astype(int)
data['isSunday'] = (data['weekday'] == 7).astype(int)

In [9]:
data_test['isWeekday'] = ((data_test['weekday'] >= 1) & (data_test['weekday'] <= 5)).astype(int)
data_test['isSaturday'] = (data_test['weekday'] == 6).astype(int)
data_test['isSunday'] = (data_test['weekday'] == 7).astype(int)

In [10]:
# weekday 삭제 
data.drop('weekday', axis=1,inplace=True)


In [11]:
data_test.drop('weekday', axis=1,inplace=True)

In [12]:
data

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,week,isWeekday,isSaturday,isSunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,1,1,0,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,1,1,0,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,1,1,0,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,9,1,0,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,9,1,0,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,9,1,0,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,9,1,0,0


In [13]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [20]:
# Encoding categorical variables and scaling
categorical_cols = ['item', 'corporation', 'location', 'year','isWeekday','isSaturday','isSunday']
numeric_features = ['month', 'day', 'week']


encoder = OneHotEncoder(sparse=False)
encoded_train = encoder.fit_transform(data[categorical_cols])
encoded_test = encoder.transform(data_test[categorical_cols])

# Correcting the column selections by adding a comma between 'day' and 'week'
train_features = np.hstack((encoded_train, data[['month', 'day', 'week']].values))
test_features = np.hstack((encoded_test, data_test[['month', 'day', 'week']].values))


In [44]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np


# Normalizing the features
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_features)

# Preparing data for LSTM
def create_dataset(dataset, target, look_back=30):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), :]
        dataX.append(a)
        dataY.append(target[i + look_back])
    return np.array(dataX), np.array(dataY)

look_back = 1
trainX, trainY = create_dataset(train_scaled, data['price(원/kg)'].values, look_back)
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[2]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, trainX.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(trainX, trainY, epochs=1000, batch_size=64, verbose=2)

# Generate predictions for test data
# Note: Properly process the test data similar to train data before prediction
test_scaled = scaler.transform(test_features)
testX, _ = create_dataset(test_scaled, np.zeros(len(test_features)), look_back)
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[2]))
test_predictions = model.predict(testX)
test_predictions = test_predictions.flatten()
# Check if we need to add a placeholder for the first prediction

sample_submission = pd.read_csv('./sample_submission.csv')

if len(test_predictions) == len(sample_submission) - 1:
    # Add a placeholder (e.g., 0 or the average of predictions) at the beginning
    placeholder = np.array([0])  # or np.mean(test_predictions)
    test_predictions = np.insert(test_predictions, 0, placeholder)

# Save predictions to submission.csv
submission = pd.DataFrame({'ID': data_test['ID'], 'answer': test_predictions})
# Update predictions where 'isSunday' is 1
# Note: Ensure that the 'isSunday' column is aligned properly with the submission DataFrame
submission.loc[data_test['isSunday'] == 1, 'answer'] = 0
submission['price(원/kg)'][submission['answer'] < 0] = 0
# Save the updated predictions to a new CSV file
submission.to_csv('1112_submission.csv', index=False)

print(submission.head())

Epoch 1/1000
929/929 - 2s - loss: 5346947.5000 - 2s/epoch - 2ms/step
Epoch 2/1000
929/929 - 1s - loss: 5250363.5000 - 761ms/epoch - 819us/step
Epoch 3/1000
929/929 - 1s - loss: 5172882.0000 - 679ms/epoch - 730us/step
Epoch 4/1000
929/929 - 1s - loss: 5101153.5000 - 674ms/epoch - 725us/step
Epoch 5/1000
929/929 - 1s - loss: 5031964.5000 - 944ms/epoch - 1ms/step
Epoch 6/1000
929/929 - 1s - loss: 4964421.5000 - 824ms/epoch - 887us/step
Epoch 7/1000
929/929 - 1s - loss: 4898357.0000 - 761ms/epoch - 819us/step
Epoch 8/1000
929/929 - 1s - loss: 4833466.5000 - 732ms/epoch - 788us/step
Epoch 9/1000
929/929 - 1s - loss: 4769712.0000 - 692ms/epoch - 745us/step
Epoch 10/1000
929/929 - 1s - loss: 4707176.5000 - 728ms/epoch - 784us/step
Epoch 11/1000
929/929 - 1s - loss: 4645720.0000 - 729ms/epoch - 785us/step
Epoch 12/1000
929/929 - 1s - loss: 4584981.0000 - 775ms/epoch - 834us/step
Epoch 13/1000
929/929 - 1s - loss: 4525224.5000 - 723ms/epoch - 778us/step
Epoch 14/1000
929/929 - 1s - loss: 446476